In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES = 3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
import pandas as pd
import torch.nn as nn

from benchmark import get_module_info

pd.set_option("display.precision", 2)

In [3]:
device = 'cuda:0'
device

'cuda:0'

In [4]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [5]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device, 
                                                               track_backward=True, memory_unit='KB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.39,0.06,0.33,0.03,33786.37,131.07,545259520.0,33280.0,1421.57
conv_64_512_3x3,0.82,0.04,1.27,0.04,36835.33,1179.65,4690151424.0,295424.0,758.6
bottleneck_64_512_3x3,0.76,0.03,1.52,0.04,37294.08,2686.98,2383218688.0,150048.0,826.01
conv_64_512_3x3_g2,1.98,0.04,2.73,0.03,34849.28,3211.26,11956733952.0,803328.0,1675.17
conv_64_512_3x3_g8,0.67,0.02,1.48,0.03,32983.04,802.82,2994898944.0,201216.0,805.66
conv_padding,0.76,0.06,0.95,0.05,36835.33,91357.18,4840226816.0,295424.0,687.05
conv_nn_Padding,0.83,0.04,1.31,0.03,41291.78,1179.65,4840226816.0,295424.0,777.62


In [6]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, memory_unit='MB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.42,0.03,0.31,0.07,38.24,35.65,545259520.0,33280.0,349.55
conv_64_512_3x3,1.0,0.03,1.37,0.04,39.03,35.65,4690151424.0,295424.0,881.7
bottleneck_64_512_3x3,0.83,0.03,2.41,0.05,41.57,35.65,2383218688.0,150048.0,1212.15
conv_64_512_3x3_g2,1.93,0.05,13.68,0.06,39.24,41.11,11956733952.0,803328.0,5445.63
conv_64_512_3x3_g8,1.09,0.04,12.21,0.05,37.37,36.45,2994898944.0,201216.0,4620.0
conv_padding,1.0,0.04,2.07,0.05,40.04,35.65,4840226816.0,295424.0,1147.23
conv_nn_Padding,1.06,0.04,1.39,0.04,44.3,35.65,4840226816.0,295424.0,942.18


In [7]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device,
                                                               track_backward=True, amp=True, memory_unit='GB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.37,0.03,0.33,0.06,0.04,0.04,545259520.0,33280.0,350.54
conv_64_512_3x3,0.48,0.03,0.52,0.09,0.04,0.04,4690151424.0,295424.0,435.78
bottleneck_64_512_3x3,0.7,0.05,1.02,0.11,0.04,0.04,2383218688.0,150048.0,718.17
conv_64_512_3x3_g2,10.71,0.06,12.2,0.04,0.04,0.04,11956733952.0,803328.0,7908.82
conv_64_512_3x3_g8,2.58,0.04,10.12,0.04,0.04,0.04,2994898944.0,201216.0,4410.42
conv_padding,0.49,0.04,0.62,0.07,0.04,0.04,4840226816.0,295424.0,477.7
conv_nn_Padding,0.56,0.09,0.62,0.08,0.04,0.04,4840226816.0,295424.0,521.11


In [8]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                       columns=['forward time mean(ms)', 'forward time std(ms)',
                                                'backward time mean(ms)', 'backward time std(ms)',
                                                'forward memory','backward memory',
                                                'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = get_module_info(module_value, inputs=shape, device=device,
                                                               track_backward=True, channels_last=True, amp=True, memory_unit='B')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.37,0.03,0.35,0.07,37980672.0,35654144.0,545259520.0,33280.0,349.14
conv_64_512_3x3,0.48,0.03,0.55,0.09,38805504.0,35654144.0,4690151424.0,295424.0,446.91
bottleneck_64_512_3x3,0.74,0.05,1.12,0.1,39493120.0,35654144.0,2383218688.0,150048.0,756.78
conv_64_512_3x3_g2,10.71,0.05,12.2,0.05,38068736.0,41105920.0,11956733952.0,803328.0,7904.62
conv_64_512_3x3_g8,2.57,0.04,10.12,0.05,36202496.0,36454400.0,2994898944.0,201216.0,4406.54
conv_padding,0.49,0.19,0.59,0.07,39845888.0,35654144.0,4840226816.0,295424.0,462.35
conv_nn_Padding,0.54,0.05,0.61,0.06,44304384.0,35654144.0,4840226816.0,295424.0,517.76
